In [730]:
import numpy as np
np.__version__

'1.22.3'

文档阅读说明：

- 🐧 表示 Tip
- ⚠️ 表示注意事项

## 广播

广播描述了NumPy在数值计算如何处理不同形状的数组。在一定限制条件下，小数组会广播大大数组以适配其形状。

In [464]:
# 最简单的例子
a = np.array([1., 2., 3.])
a

array([1., 2., 3.])

In [465]:
a * 2

array([2., 4., 6.])

In [466]:
# 上面的例子等价于
b = np.array([2, 2, 2])
a * b

array([2., 4., 6.])

广播的规则如下：

- 从右到左比较
- 兼容时相等或某个维度为1
- 数组可以不同维度

In [467]:
a = np.ones((8, 1, 6, 1))
b = np.ones((7, 1, 5))

In [468]:
# b 等价于变成了 (1, 7, 1, 5) 的shape
c = a + b
c.shape

(8, 7, 6, 5)

In [475]:
# 不能广播的例子
a = np.ones((2, 3, 4))
b = np.ones((2, 3))
a + b

ValueError: operands could not be broadcast together with shapes (2,3,4) (2,3) 

In [476]:
# 当然一维向量数字的数量就是维度
# 这样是不行的
a = np.ones((2, 3))
b = np.ones((2, ))
b.shape

(2,)

In [472]:
a + b

ValueError: operands could not be broadcast together with shapes (2,3) (2,) 

In [473]:
# 这样就可以
b = np.ones((3, ))
a + b

array([[2., 2., 2.],
       [2., 2., 2.]])

大部分时候我们可以无伤地使用它，但当数据量很大时，广播会复制数组，可能会导致内存溢出。

In [477]:
rng = np.random.default_rng(42)
a = rng.random((102400, 256, 64))
b = rng.random((102400, 256))
c = rng.random((256, 64))

In [478]:
# a=12.5G
102400*256*64*8/1024/1024/1024

12.5

In [ ]:
# 如果内存小于25G，这里会超出内存，因为差的结果还是会存在临时空间
a[:] = b[:, :, np.newaxis] - c

In [ ]:
# 可以用之前提到的 out 参数，不会额外增加内存
d = np.subtract(b[:, :, np.newaxis], c, out=a)

不过，一般会使用虚拟内存，不用过于担心。

## 通函数

通函数（ufunc）是以逐个元素的方式对ndarray进行操作的函数，支持数组广播、类型转换等。在NumPy中，通用函数都是`np.ufunc`的实例。

In [479]:
isinstance(np.add, np.ufunc)

True

`+`是`np.add`的快捷方式，其他的函数也类似：

In [481]:
x1 = np.arange(9).reshape((3, 3))
x2 = np.arange(3)

In [482]:
x1 + x2

array([[ 0,  2,  4],
       [ 3,  5,  7],
       [ 6,  8, 10]])

In [483]:
np.add(x1, x2)

array([[ 0,  2,  4],
       [ 3,  5,  7],
       [ 6,  8, 10]])

In [484]:
x1 * x2

array([[ 0,  1,  4],
       [ 0,  4, 10],
       [ 0,  7, 16]])

In [485]:
np.multiply(x1, x2)

array([[ 0,  1,  4],
       [ 0,  4, 10],
       [ 0,  7, 16]])

out 参数主要是用来存储计算结果：

In [486]:
x3 = np.zeros((3, 3), dtype=np.int8)
x3

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [487]:
np.add(x1, x2, out=x3)

array([[ 0,  2,  4],
       [ 3,  5,  7],
       [ 6,  8, 10]], dtype=int8)

In [488]:
x3

array([[ 0,  2,  4],
       [ 3,  5,  7],
       [ 6,  8, 10]], dtype=int8)

where 参数确定哪些可以存储：

In [489]:
x4 = np.zeros((3, 3), dtype=np.int8)
x4

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [490]:
np.add(x1, x2, out=x4, where=[True, False, False])

array([[0, 0, 0],
       [3, 0, 0],
       [6, 0, 0]], dtype=int8)

In [491]:
x4

array([[0, 0, 0],
       [3, 0, 0],
       [6, 0, 0]], dtype=int8)

NumPy中的ufunc数量很多，包括数学运算、三角函数、位操作、逻辑函数、浮点函数等，具体可查看：

- https://numpy.org/devdocs/reference/ufuncs.html#available-ufuncs

ufunc可以被`__array_ufunc__`方法覆盖，具体可参考第一章《核心概念：自定义数组容器》。

ufunc支持以下方法：

- reduce：沿着某个维度累积
- accumulate：所有元素累积
- reduceat：沿着某个维度指定的slice累积
- outer：对A和B中所有元素对运算
- at：对指定索引的元素的执行无缓冲的就地运算

In [492]:
a = np.arange(12).reshape(4, 3)
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

`reduce`的参数和初级课程介绍的很多接口差不多：

- array：数组
- axis：维度
- dtype：数据类型
- out：同上
- where：同上
- keepdims：是否保持维度，《基础教程》中有介绍
- initial：初始值

In [493]:
np.add.reduce(a, axis=0, initial=10)

array([28, 32, 36])

In [494]:
np.add.reduce(a, axis=1, initial=10, keepdims=True)

array([[13],
       [22],
       [31],
       [40]])

`accumulate`的参数少很多：

- array：数组
- axis：维度
- dtype：数据类型
- out：同上

In [495]:
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [496]:
# 沿着行
np.multiply.accumulate(a, axis=1)

array([[  0,   0,   0],
       [  3,  12,  60],
       [  6,  42, 336],
       [  9,  90, 990]])

In [497]:
np.multiply.accumulate(a)

array([[  0,   1,   2],
       [  0,   4,  10],
       [  0,  28,  80],
       [  0, 280, 880]])

`reduceat`相比`accumulate`增加了索引位置：

- indices：index的复数
- 其他参数同`accumulate`


计算 `array[indices[i]:indices[i+1]]`，i 表示第i行/列，计算规则如下：

- 当`i = len(indices) - 1`（最后一个index）：`indices[i+1] = array.shape[axis]`
- 当`indices[i] >= indices[i + 1]`，第i个就是 `array[indices[i]]`
- 当`indices[i] >= len(array)` 或 `indices[i] < 0`，错误

In [498]:
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [499]:
np.add.reduceat(a, [1,2])

array([[ 3,  4,  5],
       [15, 17, 19]])

In [500]:
# 四列
# 第0列：indices[0]: indices[1]，即第0:2累积（0+1列）
# 第1列：indices[1] > indices[2]，等于第2列
# 第2列：indices[2] > indices[3]，等于第1列
# 第3列：最后一个，indices[3]: indices[a.shape[1]=3]，等于0:3累积（0,1,2列）
np.add.reduceat(a, [0, 2, 1, 0], axis=1)

array([[ 1,  2,  1,  3],
       [ 7,  5,  4, 12],
       [13,  8,  7, 21],
       [19, 11, 10, 30]])

`outer`接受两个数组，等价于以下结果：

```python
r = empty(len(A),len(B))
for i in range(len(A)):
    for j in range(len(B)):
        r[i,j] = op(A[i], B[j])
```

In [501]:
np.add.outer(a, a).shape

(4, 3, 4, 3)

In [502]:
np.add.outer([1,2,3], [4,5,6])

array([[5, 6, 7],
       [6, 7, 8],
       [7, 8, 9]])

`at` 参数如下：

- a：数组
- indices：索引
- b：两个操作对象时另一个操作对象

In [503]:
a = np.arange(12).reshape(4, 3)
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [504]:
# 对第0和1行加1
np.add.at(a, [0, 1], [1])

In [505]:
a

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [506]:
# 4x3 和 1x3 可以通过广播运算
a = np.arange(12).reshape(4, 3)
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [507]:
np.add.at(a, [0, 1], np.array([[1, 2, 3]]))
a

array([[ 1,  3,  5],
       [ 4,  6,  8],
       [ 6,  7,  8],
       [ 9, 10, 11]])

可以使用`np.fromfunc`来生成通函数实例。本章后面会进一步介绍，不再赘述。

## 基本操作

## 排序搜索

### 极值

关于最大最小值的方法不少，我们看看。

`np.maximum`和`np.minimum`是通函数，以max为例，其他相关的还有`np.max`（等价的`np.amax`），`np.fmax`和`np.nanmax`。区分如下：

- `minimum`：两个数组逐元素比较
- `fmax`：同上，但忽略缺失值
- `amax`：沿着给定维度
- `nanmax`：同上，但忽略缺失值

我们以极大值为例。

In [15]:
# 等价于 np.where(x1 >= x2, x1, x2)
np.fmax([np.nan, 2, 3], [1, 5, np.nan])

array([1., 5., 3.])

In [13]:
np.nanmax([[np.nan, 3, 5], [2, 1, np.nan]], axis=0)

array([2., 3., 5.])

### 搜索

`argmax/argmin`我们在《从小白到入门》中已经做了介绍，这里主要介绍带非数值的版本，以极小值为例。

In [513]:
a = np.array([
    [np.nan, 2, 3],
    [1, np.nan, 4]
])

In [516]:
np.argmin(a, axis=0)

array([0, 1, 0])

In [517]:
np.nanargmin(a, axis=0)

array([1, 0, 0])

`argwhere`和《从小白到入门》中介绍的`where`有所不同，它返回所有非0元素的index，是`where`的弱化版本。

In [524]:
a = np.arange(6).reshape(2, 3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [526]:
# 默认返回非0的元素index
np.argwhere(a)

array([[0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2]])

In [529]:
# 这个不是指定条件，而是a>3本身是个数组
np.argwhere(a>3)

array([[1, 1],
       [1, 2]])

对于非0位置的索引，还有两个API：

In [535]:
# 返回非0元素的索引
np.nonzero(a>2)

(array([1, 1, 1]), array([0, 1, 2]))

In [537]:
# 返回打平后的索引
np.flatnonzero(a>2)

array([3, 4, 5])

最后是`searchsorted`，返回给定值应该插入的位置索引。给定的数组应为一维，但插入的值可以是数组。参数如下：

- 被插入的一维数组。
- 要插入的值，数组。
- side：默认left，第一个合适的位置；right为最后一个合适的位置。
- sorter：一维数组，可选的索引，将要插入的数组排序为升序。

In [558]:
a = np.arange(1, 6)
b = np.array([3, 2, 1, 5, 4])
a

array([1, 2, 3, 4, 5])

In [547]:
np.searchsorted(a, 2)

1

In [548]:
np.searchsorted(b, 2)

3

In [554]:
# 要插入的值可以是多维数组
np.searchsorted(a, [[2,2],[2,2]])

array([[1, 1],
       [1, 1]])

side控制在所有合适位置中插入位置的index：

In [564]:
np.searchsorted([1,1,1,1,1], 1, "right")

5

In [565]:
np.searchsorted([1,1,1,1,1], 1, "left")

0

sorter表示被插入数组元素的索引，该索引将数组按升序排列。

In [578]:
b

array([3, 2, 1, 5, 4])

In [579]:
np.searchsorted(b, 2)

3

In [583]:
# 1 2 3 4 5
np.searchsorted(b, 2, sorter=[2,1,0,4,3])

1

In [587]:
np.searchsorted([1,2,3,4,5],2)

1

In [585]:
# 5 4 3 2 1
np.searchsorted(b, 2, sorter=[3,4,0,1,2])

0

In [586]:
np.searchsorted([5,4,3,2,1],2)

0

### 排序

`argsort`在《从小白到入门》中已有介绍，此处不再赘述。这里介绍其他几个和排序相关的API：

- `sort/lexsort/ndarray.sort`：数组排序
- `msort`：排序第一轴
- `sort_complex`：复数排序
- `prtition/argpartition/ndarray.partition`：部分排序

In [633]:
rng = np.random.default_rng(42)
a = rng.integers(0, 10, (3, 4))
a

array([[0, 7, 6, 4],
       [4, 8, 0, 6],
       [2, 0, 5, 9]])

`sort`包括以下参数：

- 数组
- axis：默认最后一个轴（-1）
- kind：kind是排序算法，支持：quicksort、mergesort、heapsort和stable，默认quicksort
- order：用来排序的field（数组有field时）

沿着最后一个维度（默认）排序时不创建临时复制，因此速度最快，不占用额外空间。

stable会根据被排序的数据类型自动选择最稳定的排序算法。

In [634]:
# 默认axis=-1
np.sort(a)

array([[0, 4, 6, 7],
       [0, 4, 6, 8],
       [0, 2, 5, 9]])

In [635]:
# 指定轴
np.sort(a, axis=0)

array([[0, 0, 0, 4],
       [2, 7, 5, 6],
       [4, 8, 6, 9]])

In [636]:
# 指定排序算法
np.sort(a, kind="stable")

array([[0, 4, 6, 7],
       [0, 4, 6, 8],
       [0, 2, 5, 9]])

In [637]:
# 指定order
# String, float16, int32
dtype = [("name", "U10"), ("height", "f2"), ("age", "i4")]
values = [
    ("Arthur", 1.8, 41), 
    ("Lancelot", 1.9, 38), 
    ("Galahad", 1.7, 38)
]
b = np.array(values, dtype=dtype)
np.sort(b, order=["height"])

array([('Galahad', 1.7, 38), ('Arthur', 1.8, 41), ('Lancelot', 1.9, 38)],
      dtype=[('name', '<U10'), ('height', '<f2'), ('age', '<i4')])

`lexsort`，后面的key优先。

In [638]:
surnames =    ('Zertz',    'Halilei', 'Halilei')
first_names = ('Heinrich', 'Gzlileo', 'Gustav')
# 先surname，在按firstname
ind = np.lexsort((first_names, surnames))
ind

array([2, 1, 0])

In [639]:
[surnames[i] + ", " + first_names[i] for i in ind]

['Halilei, Gustav', 'Halilei, Gzlileo', 'Zertz, Heinrich']

`ndarray.sort`是in-place排序。其余参数与`np.sort`一样。

In [645]:
rng = np.random.default_rng(42)
a = rng.integers(0, 10, (3, 4))
a

array([[0, 7, 6, 4],
       [4, 8, 0, 6],
       [2, 0, 5, 9]])

In [646]:
a.sort()

In [647]:
a

array([[0, 4, 6, 7],
       [0, 4, 6, 8],
       [0, 2, 5, 9]])

`msort`与`np.sort(a, axis=0)`等价。

In [650]:
rng = np.random.default_rng(42)
a = rng.integers(0, 10, (3, 4))
a

array([[0, 7, 6, 4],
       [4, 8, 0, 6],
       [2, 0, 5, 9]])

In [651]:
np.msort(a)

array([[0, 0, 0, 4],
       [2, 7, 5, 6],
       [4, 8, 6, 9]])

`sort_complex`先用实部排，再用虚部。

In [657]:
np.sort_complex([1 + 2j, 1+1j, 2 - 1j, 3 - 2j, 3 - 3j, 3 + 5j])

array([1.+1.j, 1.+2.j, 2.-1.j, 3.-3.j, 3.-2.j, 3.+5.j])

`partition`和`argpartition`的关系和`sort`与`argsort`类似。参数包括：

- 数组
- kth：切分的位置，整数或整数序列
- axis：轴，默认-1
- kind：选择算法，默认`introselect`
- order：str或List[str]，和前面介绍的`sort`中的参数用法一样。

注意，返回的结果中，第k个元素的位置是排好序时的位置（不是原始数组中的index）。

In [723]:
a = [100, 99, 87, 101, 88, 78, 98]
# 88 排好时在index=2的数字
# 比88小的在左边，比88大或相等的在右边
np.partition(a, 2)

array([ 78,  87,  88, 101,  99, 100,  98])

In [724]:
np.argpartition(a, 2)

array([5, 2, 4, 3, 1, 0, 6])

In [725]:
a = [3, 4, 2, 1]
# 4 是排好序时index=3的数字
np.partition(a, 3)

array([2, 1, 3, 4])

In [726]:
np.argpartition(a, 3)

array([2, 3, 0, 1])

In [727]:
rng = np.random.default_rng(42)
a = rng.integers(0, 10, (3, 8))
a

array([[0, 7, 6, 4, 4, 8, 0, 6],
       [2, 0, 5, 9, 7, 7, 7, 7],
       [5, 1, 8, 4, 5, 3, 1, 9]])

In [728]:
# 注意第一行，比4大或相等的在右边 
np.partition(a, 2)

array([[0, 0, 4, 6, 4, 8, 7, 6],
       [0, 2, 5, 9, 7, 7, 7, 7],
       [1, 1, 3, 4, 5, 8, 5, 9]])

In [729]:
np.argpartition(a, 2)

array([[0, 6, 3, 2, 4, 5, 1, 7],
       [1, 0, 2, 3, 4, 5, 6, 7],
       [1, 6, 5, 3, 4, 2, 0, 7]])

## 集合操作

第一个数组的元素是否在第二个数组：

In [192]:
a = np.array([[1, 2], [2, 3]])
np.unique(a)

array([1, 2, 3])

In [193]:
np.in1d([1,2,3,4], a)

array([ True,  True,  True, False])

In [194]:
# 翻转
np.in1d([1,2,3,4], a, invert=True)

array([False, False, False,  True])

In [195]:
# 两个数组里的值均unique
# 可以加速计算
np.in1d([1,2,3,4], a, assume_unique=True)

array([ True,  True,  True, False])

In [200]:
# 打平
np.in1d([[1,2],[3,4]], a)

array([ True,  True,  True, False])

In [201]:
# 不打平
np.isin([1,2,3,4], a)

array([ True,  True,  True, False])

In [198]:
np.isin([[1,2],[3,4]], a)

array([[ True,  True],
       [ True, False]])

其余参数和`in1d`一样。

交集，输入的数组会打平：

In [210]:
a

array([[1, 2],
       [2, 3]])

In [211]:
b = np.array([[2, 4, 1]])

In [212]:
np.intersect1d(a, b)

array([1, 2])

In [213]:
# 如果假设为unique，其实不是，结果会有误
np.intersect1d(a, b, assume_unique=True)

array([1, 2, 2])

In [214]:
# 返回索引，如果有多个，返回第一个
np.intersect1d(a, b, return_indices=True)

(array([1, 2]), array([0, 1]), array([2, 0]))

并集，输入的数组会打平：

In [215]:
a

array([[1, 2],
       [2, 3]])

In [218]:
b

array([[2, 4, 1]])

In [216]:
np.union1d(a, b)

array([1, 2, 3, 4])

差集，返回在数组1不在数组2的：

In [219]:
np.setdiff1d(a, b)

array([3])

In [220]:
np.setdiff1d(b, a)

array([4])

异或集：

In [224]:
np.setxor1d(a, b)

array([3, 4])

对于多个数组，可以使用`reduce`方法：

In [206]:
from functools import reduce

In [208]:
reduce(
    np.union1d, 
    ([1, 3, 4, 3], [3, 1, 2, 1], [6, 3, 4, 2])
)

array([1, 2, 3, 4, 6])

## 函数式编程

NumPy的向量化计算也可以被自定义使用。下面我们就看看如何在NumPy上使用自定义方法。

`apply_along_axis`是沿着某个维度应用一个函数，等于是沿着某个维度对原数组值进行映射。

In [230]:
def func(a):
    return (a[0] + a[-1]) / 2

In [232]:
b = np.arange(1, 10).reshape(3, 3)
b

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [233]:
np.apply_along_axis(func, 0, b)

array([4., 5., 6.])

In [236]:
np.apply_along_axis(func, 1, b)

array([2., 5., 8.])

In [242]:
b = np.array([[8,1,7], [4,3,9], [5,2,6]])
b

array([[8, 1, 7],
       [4, 3, 9],
       [5, 2, 6]])

In [240]:
np.apply_along_axis(sorted, 1, b)

array([[1, 7, 8],
       [3, 4, 9],
       [2, 5, 6]])

In [245]:
np.apply_along_axis(sorted, 0, b)

array([[4, 1, 6],
       [5, 2, 7],
       [8, 3, 9]])

In [253]:
np.apply_along_axis(np.diag, 1, b)

array([[[8, 0, 0],
        [0, 1, 0],
        [0, 0, 7]],

       [[4, 0, 0],
        [0, 3, 0],
        [0, 0, 9]],

       [[5, 0, 0],
        [0, 2, 0],
        [0, 0, 6]]])

In [254]:
np.apply_along_axis(np.diag, 0, b).T

array([[[8, 0, 0],
        [0, 4, 0],
        [0, 0, 5]],

       [[1, 0, 0],
        [0, 3, 0],
        [0, 0, 2]],

       [[7, 0, 0],
        [0, 9, 0],
        [0, 0, 6]]])

`apply_over_axes`会将一个函数重复多次应用到多个轴。

注意：函数接受两个参数（和`apply_along_axis`的一个参数区别），分别是数组和维度。

In [268]:
b

array([[8, 1, 7],
       [4, 3, 9],
       [5, 2, 6]])

In [266]:
np.apply_over_axes(np.sum, b, 1)

array([[16],
       [16],
       [13]])

In [267]:
np.apply_over_axes(np.sum, b, 0)

array([[17,  6, 22]])

In [265]:
np.apply_over_axes(np.sum, b, [0, 1])

array([[45]])

`vectorize`是通用类，将自定义函数向量化，主要是方便，而不是为了提高性能，内部实际是一个for循环。

In [270]:
def func(a, b):
    if a > b:
        return a - b
    else:
        return a + b

In [271]:
vfunc = np.vectorize(func)

In [272]:
vfunc(np.arange(10), 3)

array([3, 4, 5, 6, 1, 2, 3, 4, 5, 6])

可以指定输出的类型：

In [274]:
vfunc = np.vectorize(func, otypes=[np.float16])

In [275]:
vfunc(np.arange(10), 3)

array([3., 4., 5., 6., 1., 2., 3., 4., 5., 6.], dtype=float16)

或者指定不进行向量化的参数：

In [290]:
def func(a, b):
    res = np.zeros_like(b)
    for i in a:
        res += i * b
    return res

In [291]:
func([1,2,3], np.array([2,4]))

array([12, 24])

In [305]:
vfunc = np.vectorize(func, excluded=["a"])

In [306]:
vfunc(a=[1,2,3], b=[2,4])

array([12, 24])

或事后指定：

In [328]:
vfunc = np.vectorize(func)
# 第一个参数排除
vfunc.excluded.add(0)

In [329]:
vfunc([1,2,3], [2,4])

array([12, 24])

如果输入的参数不能直接执行，则需要签名：

In [426]:
conv = np.vectorize(np.convolve, signature="(n),(m)->(k)")

In [427]:
conv([[1,2,3],[4,5,6],[7,8,9]], [1, 0.5])

array([[ 1. ,  2.5,  4. ,  1.5],
       [ 4. ,  7. ,  8.5,  3. ],
       [ 7. , 11.5, 13. ,  4.5]])

In [428]:
np.convolve([1,2,3,4,5,6,7,8,9], [1, 0.5])

array([ 1. ,  2.5,  4. ,  5.5,  7. ,  8.5, 10. , 11.5, 13. ,  4.5])

相比`vectorize`，`frompyfunc`更加通用，它可以将任意的Python函数转为通函数。

In [433]:
def func(x):
    return str(x)

In [439]:
ufunc = np.frompyfunc(func, 1, 1)

In [441]:
ufunc(np.arange(5))

array(['0', '1', '2', '3', '4'], dtype=object)

`piecewise`会执行一组条件和对应的函数，当条件为真时执行该函数。

In [447]:
a = np.arange(10)

In [448]:
np.piecewise(a, [a<5, a==5, a>5], [-1, 0, 1])

array([-1, -1, -1, -1, -1,  0,  1,  1,  1,  1])

In [453]:
def func(a, b):
    return a + b

In [463]:
# 注意后面的参数是所有函数通用的
np.piecewise(a, [a<5, a==5, a>5], [func, 0, lambda x, b: x**2], b=5)

array([ 5,  6,  7,  8,  9,  0, 36, 49, 64, 81])

## 测试

## 小结

## 参考